In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
import time
driver.get("https://pser.punjab.gov.pk/login")
driver.find_element(By.XPATH,'//form[@method="POST"]//input[@name="username"]').send_keys('3110312304391')
driver.find_element(By.XPATH,'//form[@method="POST"]//input[@name="password"]').send_keys('Akhtar@1234')
driver.find_element(By.CLASS_NAME, "loginPserBtn").click()

# chatgpt code
second_div = driver.find_element(By.XPATH, '(//div[@class="col-lg-6 col-xl-6 py-2"])[2]')
second_div.click()
option = driver.find_element(By.XPATH, '//select[@class="form-select form-select-sm"]/option[@value="100"]')
option.click()


In [9]:


data = []

while True:
    # Scrape table data
    rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
    for row in rows:
        tds = row.find_elements(By.TAG_NAME, "td")
        name = tds[2].text
        cnic = tds[3].text
        phone = tds[4].text.strip()
        if not phone.startswith("92"):
             phone = "92" + phone.lstrip("0")
             data.append([phone,name, cnic ])
    
    # Try to click the "Next" button
    try:
        next_btn = driver.find_element(By.XPATH, '//li[@class="paginate_button page-item next"]')
        next_class = next_btn.get_attribute("class")

        if "disabled" in next_class:
                break

    # Scroll and click with JavaScript
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_btn)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", next_btn)

        time.sleep(2)  # Wait for next page to load
    except (NoSuchElementException, ElementNotInteractableException):
        break

# Save to CSV
with open("all_pages_data.csv", mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Phone","Name", "CNIC", ])
    writer.writerows(data)

print("✅ Data scraping completed and saved to all_pages_data.csv")




# Save to second CSV with name_cnic format
with open("name_cnic_phone.csv", mode="w", newline='', encoding='utf-8') as file2:
    writer2 = csv.writer(file2)
    writer2.writerow(["name_cnic", "phone"])
    for row in data:
        phone = row[0].strip().lower().replace(" ", "_")  # optional: lowercase + underscores
        name = row[1].strip()
        cnic = row[2]
        name_cnic = f"{name}_{cnic}"
        writer2.writerow([name_cnic, phone])

print("✅ name_cnic_phone.csv file saved successfully.")


✅ Data scraping completed and saved to all_pages_data.csv
✅ name_cnic_phone.csv file saved successfully.
